In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


import seaborn as sns
import random

# import standard scalaer and PCA
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
# import random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


In [2]:
# Import gene_expression_data
gene_expression_data = pd.read_csv('../data/gene_expression_data.csv', index_col=0)

gene_expression_data = gene_expression_data.T

In [4]:
# print 100 columns
gene_expression_data


,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
GSM7500908_S001.Cel.gz,2.977199,2.564554,2.765110,3.209871,1.867418,2.962612,2.599297,2.487776,2.580259,2.303332,...,3.549539,3.436805,3.691967,3.762129,2.357800,2.240489,2.012306,1.819301,1.971870,2.005257
GSM7500909_S002.Cel.gz,2.830919,2.406084,2.716011,3.243258,1.864083,2.853416,2.647998,2.476042,2.275419,2.273596,...,3.354232,3.238425,3.612543,3.632145,2.338294,2.190432,2.047211,1.774937,1.927410,1.987641
GSM7500910_S003.Cel.gz,2.973773,2.646121,2.779942,3.201516,1.923635,3.036835,2.633980,2.422082,2.613706,2.312740,...,3.617943,3.490235,3.758465,3.781940,2.288388,2.103839,1.971929,1.815607,1.921539,1.905453
GSM7500911_S004.Cel.gz,2.965772,2.683993,2.764771,3.211044,1.946121,3.005431,2.632986,2.477094,2.280408,2.369711,...,3.608143,3.519557,3.771382,3.782085,2.352327,2.191886,2.015793,1.802133,1.941525,2.036957
GSM7500912_S005.Cel.gz,2.907473,2.541508,2.848162,3.201331,1.893006,2.841838,2.653494,2.490366,2.152633,2.263958,...,3.591051,3.493405,3.794466,3.793630,2.423620,2.235883,2.035061,1.782187,1.943881,2.086832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM7501261_P035.Cel.gz,2.940429,2.718132,2.704712,3.054938,1.834628,3.074088,2.723030,2.274356,2.640175,2.329295,...,3.708131,3.623267,3.789973,3.800004,2.286468,2.262799,1.940855,1.834753,1.917734,1.989553
GSM7501262_P036.Cel.gz,2.971834,2.786440,2.712130,3.007769,1.816625,3.182329,2.614366,2.397496,2.797520,2.210926,...,3.593783,3.491055,3.726196,3.723939,2.325430,2.161119,2.077009,1.861306,1.920578,1.920899
GSM7501263_P037.Cel.gz,2.985284,2.846095,2.699143,3.050060,1.862027,3.185931,2.662524,2.307475,3.227731,2.348722,...,3.630801,3.531366,3.724474,3.740654,2.324892,2.344585,2.004962,1.893372,1.965883,1.941255
GSM7501264_P038.Cel.gz,2.887022,2.771997,2.588294,3.072207,1.877833,3.045920,2.517662,2.300196,3.014543,2.430810,...,3.641440,3.524972,3.736621,3.740779,2.345919,2.193342,1.985651,1.809090,1.971777,1.926345


In [5]:

# add column named state with progressing or stable if row contains _P or _S
for index, row in gene_expression_data.iterrows():
    if '_P' in index:
        gene_expression_data.loc[index, 'state'] = 1
    elif '_S' in index:
        gene_expression_data.loc[index, 'state'] = 0


## Flags for changing setting

- `no-minsampling` : Disable min-sampling
- `no-log2` : Disable log2

In [6]:
NO_MINSAMPLING = False
NO_LOG2 = False
MIN = False

In [7]:
if NO_LOG2 == False:
    gene_expression_data = np.log2(gene_expression_data + 1)

In [8]:

# set random seed
random.seed(90)



# undersample the data
# get the number of rows with state 0
state_0 = gene_expression_data[gene_expression_data['state'] == 0]
state_1 = gene_expression_data[gene_expression_data['state'] == 1]

# get the number of rows with state 1
state_0_count = state_0.shape[0]
state_1_count = state_1.shape[0]


if MIN:
    # get the minimum number of rows
    min_count = min(state_0_count, state_1_count)

    # get the same number of rows for each state
    state_0 = state_0.sample(min_count)
    state_1 = state_1.sample(min_count)

# concatenate the two dataframes
gene_expression_data = pd.concat([state_0, state_1])


# shuffle rows of gene_expression_data

#gene_expression_data = shuffle(gene_expression_data, random_state=90)


#train_data.shape, test_data.shape # 50 - 14 - 14 split
#train_data = gene_expression_data.iloc[:50, :]
#valid_data = gene_expression_data.iloc[50:64, :]
# Rest to the test data
#test_data = gene_expression_data.iloc[64:, :]

# take 70% for training and 70  for testing
train_data = gene_expression_data.iloc[:gene_expression_data.shape[0] * 70 // 100, :]
test_data = gene_expression_data.iloc[:gene_expression_data.shape[0] * 30 // 100, :]
valid_data = test_data



""" train_data = gene_expression_data.iloc[:220, :]
valid_data = gene_expression_data.iloc[220:280, :]
# Rest to the test data
test_data = gene_expression_data.iloc[300:, :]
 """
#shape
# count state in the three datasets

    

# get the train and valid data
X_train = train_data
y_train = train_data['state']

X_valid = valid_data
y_valid = valid_data['state']

In [9]:
from sklearn.model_selection import LeaveOneOut

In [10]:
gene_list = ["234764_x_at", "211835_at", "1561937_x_at", "202716_at",
                  "235305_s_at", "210538_s_at", "237461_at", "41660_at", "217892_s_at", 
                  "225822_at", "57532_at", "213489_at", "222641_s_at", "205159_at",
                  "209012_at", "220522_at", "223709_s_at", "36129_at",
                  "201848_s_at", "212704_at", "213622_at", "232531_at", 
                  "205666_at", "210789_x_at", "217809_at", "225291_at", 
                  "226488_at", "231131_at", "238662_at", "226098_at", "202387_at",
                  "228217_s_at", "225553_at", "223995_at", "202613_at", "203200_s_at"]



In [11]:
gene_expression_data[gene_list+['state']]

,234764_x_at,211835_at,1561937_x_at,202716_at,235305_s_at,210538_s_at,237461_at,41660_at,217892_s_at,225822_at,...,231131_at,238662_at,226098_at,202387_at,228217_s_at,225553_at,223995_at,202613_at,203200_s_at,state
GSM7500908_S001.Cel.gz,2.248920,2.041002,2.167790,1.969024,1.888946,2.126404,1.844011,1.887947,1.909556,1.969950,...,1.740422,1.730948,1.905067,1.938790,1.932247,1.883278,1.771093,1.803417,1.958316,0.0
GSM7500909_S002.Cel.gz,2.207382,1.978091,2.157067,2.003837,1.932545,1.938255,1.887837,1.836200,1.970447,1.944984,...,1.787242,1.737519,1.823735,1.956539,1.891698,1.901337,1.690449,1.806487,1.991732,0.0
GSM7500910_S003.Cel.gz,2.249299,2.030639,2.197212,2.049668,1.891676,2.113966,1.763531,1.816652,2.009154,1.943350,...,1.674132,1.761139,1.898302,1.993271,1.928321,1.913908,1.766920,1.715628,2.025509,0.0
GSM7500911_S004.Cel.gz,2.221557,2.032188,2.123498,2.034761,1.883183,2.142557,1.775918,1.838905,1.977184,1.966631,...,1.712760,1.765829,1.890841,2.062605,2.000059,1.916128,1.814050,1.850190,1.965160,0.0
GSM7500912_S005.Cel.gz,2.234426,1.948068,2.071428,1.990820,1.909897,2.052060,1.851227,1.836886,1.948224,1.921008,...,1.935572,1.776754,1.914654,2.074969,2.010105,1.927738,1.824873,1.852655,2.023261,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSM7501261_P035.Cel.gz,2.252009,2.045332,2.123731,2.081890,1.863696,2.185876,1.905585,1.814033,1.904506,1.925355,...,1.685821,1.867867,2.026572,2.054701,2.044580,1.965256,1.873525,1.846187,2.057208,1.0
GSM7501262_P036.Cel.gz,2.150930,1.972151,1.959934,2.086806,1.843420,2.171185,1.780236,1.805270,1.910756,1.936274,...,1.811833,1.924688,2.038862,2.085349,2.051529,1.984635,1.989656,1.889096,2.073161,1.0
GSM7501263_P037.Cel.gz,2.224666,2.046958,2.179468,2.084121,1.839278,2.154838,1.862945,1.833517,1.961075,1.919762,...,1.895629,1.907112,2.025385,2.105732,2.084471,2.040043,1.985128,1.956334,2.080250,1.0
GSM7501264_P038.Cel.gz,2.195128,1.970426,2.145159,2.070294,1.854169,2.092897,1.615010,1.779359,1.863141,1.791136,...,2.067156,1.964567,2.038066,2.112278,2.136149,2.085936,1.991955,1.973411,2.094743,1.0


In [12]:
cv = LeaveOneOut()

# taking the train split that we have done at the beginning
X = gene_expression_data
# from X take only selected_features_final
X = X.loc[:, gene_list + ['state']]

y_true = []
y_pred = []

for train_ix, valid_ix in cv.split(X):

    X_train, X_valid = X.iloc[train_ix], X.iloc[valid_ix]

    y_train, y_valid = X_train['state'], X_valid['state']

    X_train = X_train.drop('state', axis=1)
    X_valid = X_valid.drop('state', axis=1)
    
    # use random forest
    rf = RandomForestClassifier(n_estimators=300, max_depth=100, random_state=4)
    print("Iteration: ", len(y_true) + 1)
     
    rf.fit(X_train, y_train)
    
    #evaluate
    yhat = rf.predict(X_valid)
    
    y_true.append(y_valid.values[0])
    y_pred.append(yhat[0])
    
accuracy = np.mean(np.array(y_true) == np.array(y_pred))
print("Accuracy: ", accuracy)
# chekc also recall and F1
from sklearn.metrics import recall_score, f1_score

recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print("Recall: ", recall)
print("F1: ", f1)

Accuracy:  0.8854748603351955
Recall:  0.07692307692307693
F1:  0.12765957446808512


# First Run

Accuracy:  0.8854748603351955
Recall:  0.07692307692307693
F1:  0.12765957446808512

In [ ]:

probeset_names = ["234764_x_at", "211835_at", "1561937_x_at", "202716_at",
                  "235305_s_at", "210538_s_at", "237461_at", "41660_at", "217892_s_at", 
                  "225822_at", "57532_at", "213489_at", "222641_s_at", "205159_at",
                  "209012_at", "220522_at", "223709_s_at", "36129_at",
                  "201848_s_at", "212704_at", "213622_at", "232531_at", 
                  "205666_at", "210789_x_at", "217809_at", "225291_at", 
                  "226488_at", "231131_at", "238662_at", "226098_at", "202387_at",
                  "228217_s_at", "225553_at", "223995_at", "202613_at", "203200_s_at"]

## Fitness function proposal for unbalanced data


$$
F_{\text{proposed}} = \frac{\sum_{\text{majority}} |\text{dist}_{\text{major}_i}|^2}{2 \times N_{\text{majority}}} + \frac{\sum_{\text{minority}} |\text{dist}_{\text{minor}_i}|^2}{2 \times N_{\text{minority}}}
$$

Where
- $N_{\text{majority}}$ is the number of samples in the majority class
- $N_{\text{minority}}$ is the number of samples in the minority class
- $\text{dist}_{\text{major}_i}$ is the distance of prediction of the $i$-th sample in the majority class to the decision boundary
- $\text{dist}_{\text{minor}_i}$ is the distance of prediction of the $i$-th sample in the minority class to the decision boundary

In [ ]:
# How to we measure the distance between the data points when doing a binary classification?
# Assuming the data are scaled, we can use the euclidean distance



In [ ]:
def train_and_evaluate_rf(train_data, valid_data, selected_genes_set):
    
    # get the selected genes and state
    temp_train_data = train_data.loc[:, selected_genes_set]
    
    # train the model
    rf = RandomForestClassifier(n_estimators=300, max_depth=100, random_state=120)
    rf.fit(temp_train_data, y_train)
    
    # evaluate the model
    accuracy = rf.score(X_valid.loc[:, selected_genes_set], y_valid)
    
    return accuracy

            

In [ ]:
def train_and_evaluate_SVC(train_data, valid_data, selected_genes_set):


    # get the selected genes and state
    temp_train_data = train_data.loc[:, selected_genes_set]
    
    # train the model
    svc = SVC(kernel='linear', C=1)
    svc.fit(temp_train_data, y_train)
    
    # evaluate the model
    accuracy = svc.score(X_valid.loc[:, selected_genes_set], y_valid)
    
    # for each of the datapoint compute the distacnce to the hyperplane
    # get the support vectors
    support_vectors = svc.support_vectors_
    # get the coefficients
    coef = svc.coef_
    
    # get the distance of each point to the hyperplane
    distance_minority_class = []
    distance_majority_class = []
    
    for index, row in X_valid.iterrows():
        # get the distance of the point to the hyperplane
        distance = np.dot(coef, row[selected_genes_set].values)
        if y_valid.loc[index] == 0:
            distance_majority_class.append(distance)
        else:
            distance_minority_class.append(distance)

    return accuracy, distance_minority_class, distance_majority_class

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.svm import SVC

def fscore(maj_distances, min_distances):
    # print(len(maj_distances),len(min_distances))
    return (sum(maj_distances)**2 / 2*len(maj_distances)) + ((sum(min_distances)**2) / 2*len(min_distances))
    
    
def train_and_evaluate_SVC_kfold(data, selected_genes_set, n_splits=4):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    accuracies = []
    scores=[]

    for train_index, valid_index in skf.split(data[selected_genes_set], data['state']): # replace 'target_column' with the name of your target column
        X_train, X_valid = data.iloc[train_index], data.iloc[valid_index]
        y_train, y_valid = X_train['state'], X_valid['state'] # replace 'target_column' with the name of your target column
        
        # get the selected genes
        temp_X_train = X_train.loc[:, selected_genes_set]
        temp_X_valid = X_valid.loc[:, selected_genes_set]
        
        # train the model
        svc = SVC(kernel='linear', C=10)
        svc.fit(temp_X_train, y_train)
        
        # evaluate the model
        accuracy = svc.score(temp_X_valid, y_valid)
        accuracies.append(accuracy)
        
        # for each datapoint compute the distance to the hyperplane
        # get the support vectors
        support_vectors = svc.support_vectors_
        # get the coefficients
        coef = svc.coef_
        
        # get the distance of each point to the hyperplane
        distance_minority_class = []
        distance_majority_class = []
        
        for index, row in temp_X_valid.iterrows():
            # get the distance of the point to the hyperplane
            distance = np.dot(coef, row.values)[0]
            if y_valid.loc[index] == 0:  # Assuming minority class is labeled as 0
                distance_majority_class.append(distance)
            else:
                distance_minority_class.append(distance)
        scores.append(fscore(distance_majority_class,distance_minority_class))
    
    

    return np.mean(accuracies), np.mean(scores)


In [ ]:
train_and_evaluate_SVC_kfold(train_data, probeset_names)

In [ ]:
# select only list of genes from the data
all_genes = gene_expression_data.columns[:-1]

# convert to a list
all_genes = all_genes.tolist()

In [ ]:
import pygad 

# let's try to use pygad for selecting features (in this case our genes in gene_expression_data),
# we know that the following probeset_names are quite good for classification

# we are doing a classification problem, so we can use the following probeset_names as initial features and
# then add more from the dataset

probeset_names = ["234764_x_at", "211835_at", "1561937_x_at", "202716_at",
                  "235305_s_at", "210538_s_at", "237461_at", "41660_at", "217892_s_at", 
                  "225822_at", "57532_at", "213489_at", "222641_s_at", "205159_at",
                  "209012_at", "220522_at", "223709_s_at", "36129_at",
                  "201848_s_at", "212704_at", "213622_at", "232531_at", 
                  "205666_at", "210789_x_at", "217809_at", "225291_at", 
                  "226488_at", "231131_at", "238662_at", "226098_at", "202387_at",
                  "228217_s_at", "225553_at", "223995_at", "202613_at", "203200_s_at"]


# try to search over all the possible sets using pygad by starting from probeset_names

# Fitness function used in the paper: A novel fitness function
# in genetic programming for medical data classification

import numpy as np
import pygad
from numpy.random import RandomState

random_seed = 1234
state = RandomState(random_seed)

def fitness_function(ga_instance, solution, solution_idx):
    
    
    mask = np.zeros(len(all_features), dtype=bool)
    
    solution_indices = np.where(solution)[0]
    mask[solution_indices] = 1
    selected_features = np.array(all_features)[mask].tolist()
    
    """
    # change mask such that we mask solution out of all_features
    mask = np.zeros(len(all_features))
    # Set to 1 only the indexes where solution is 1
    mask[solution == 1] = 1
   
    # Apply the mask
    selected_features = np.array(all_features)[mask.astype(bool)].tolist()
    """
    
    num_selected_genes = 100
  
    # Create a random mask selecting out of all_features only num_selected_genes then apply the mask and take only 
    # the selected genes into a list
    
    # 1. Random mask
    #mask = np.zeros(len(all_features))  # Create a mask of all False
    #mask[:num_selected_genes] = 1  # Set the first 'num_features' elements to True
    # shuffle the mask
    #np.random.shuffle(mask)
    
    # 2. Apply the mask
    #selected_features =  np.array(all_features)[mask.astype(bool)].tolist()
    
    #print(selected_features) 
     
    
    # Train a model using the subset of genes and evaluate its performance
    # You would need to replace this with your own model training and evaluation code
    
    #accuracy = train_and_evaluate_rf(train_data=train_data, valid_data=valid_data, selected_genes_set=selected_features)
    accuracy, score = train_and_evaluate_SVC_kfold(data=train_data, selected_genes_set=selected_features)
    

    return score

# Step 2: Define your gene list
gene_list = probeset_names

# remove the genese in gene_list from the dataframe
remaining_genes = gene_expression_data.drop(gene_list, axis=1)

remaining_genes_with_state = remaining_genes.copy()
remaining_genes = remaining_genes.drop('state', axis=1)

all_features = remaining_genes.columns.tolist()

def on_start(ga_instance):
    print("on_start")

def on_fitness(ga_instance, last_gen_fitness):
    print("on_fitness")

def on_generation(ga_instance):
   
   print("on_generation number {0}".format(ga_instance.generations_completed))
   print("Best solution fitness: {0}".format(ga_instance.best_solution()[1]))
    
    
def on_stop(ga_instanse, last_gen_fitness):
    print("on_stop")


# put this to 100 will search in the first 100
added_genes = len(all_features)

# probset names + added_genes is the len of the subset we are going to use
m = added_genes

# number of genes to select
num_genes = 80

# init with random subset of genes in total_genes
gene_space = state.random_integers(0, 1, m)

parent_selection_type = "sss"
keep_parents = 2

crossover_type = "single_point"

mutation_type = "random"
mutation_percent_genes = 20

# Step 3: Create an instance of the pygad.GA class
ga_instance = pygad.GA(gene_space=gene_space,
                       num_generations=10,
                       num_parents_mating=30,
                       fitness_func=fitness_function,
                       sol_per_pop=30,
                       num_genes=num_genes,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_percent_genes=mutation_percent_genes,
                       gene_type=float,
                       on_start=on_start,
                       on_fitness=on_fitness,
                       on_generation=on_generation,
                       on_stop=on_stop,
                       random_seed=random_seed) 

                       

In [ ]:
# Step 4: Run the GA
ga_instance.run()

ga_instance.plot_fitness()
# Step 5: Get the best solution and its fitness value
solution, best_solution_fitness, best_match_idx  = ga_instance.best_solution()
# get the best subset by taking the solution with 80 genes
#best_gene_subset = [gene_list[i] for i, gene in enumerate(solution) if np.any(gene == 1)]


print("Best solution: ", solution)
print("Best solution fitness: ", best_solution_fitness)
print("Best solution index: ", best_match_idx)

#print("Best subset of genes:", best_gene_subset)



In [ ]:
# retrieve the best solution 
mask = np.where(solution == 1)[0]
selected_features = np.array(all_features)[mask]

print("Selected features: ", selected_features)

In [ ]:
# get selected features and train from scratch another model to evaluate its performance
selected_features_final = selected_features.tolist() + gene_list



## Trying Random Forest on all features

In [ ]:
from sklearn.model_selection import LeaveOneOut 

In [ ]:
# Get data using only selected_features_final columns
train_data_final = train_data.loc[:, selected_features_final]
valid_data_final = valid_data.loc[:, selected_features_final]
test_data_final = test_data.loc[:, selected_features_final]

# only selected
train_data_selected = train_data.loc[:, selected_features]
valid_data_selected = valid_data.loc[:, selected_features]
test_data_selected = test_data.loc[:, selected_features]


# Leave one out cross validation

In [ ]:
selected_features_final


# make selected_features a list
#selected_features = selected_features.tolist()

#len(gene_list)
len(selected_features_final)
#len(selected_features)




In [ ]:
cv = LeaveOneOut()

# taking the train split that we have done at the beginning
X = train_data
# from X take only selected_features_final
X = X.loc[:, gene_list + ['state']]

y_true = []
y_pred = []

for train_ix, valid_ix in cv.split(X):

    X_train, X_valid = X.iloc[train_ix], X.iloc[valid_ix]

    y_train, y_valid = X_train['state'], X_valid['state']

    X_train = X_train.drop('state', axis=1)
    X_valid = X_valid.drop('state', axis=1)
    
    # use random forest
    rf = RandomForestClassifier(n_estimators=300, max_depth=100, random_state=4)
    
    rf.fit(X_train, y_train)
    
    #evaluate
    yhat = rf.predict(X_valid)
    
    y_true.append(y_valid.values[0])
    y_pred.append(yhat[0])
    
accuracy = np.mean(np.array(y_true) == np.array(y_pred))
print("Accuracy: ", accuracy)
# chekc also recall and F1
from sklearn.metrics import recall_score, f1_score

recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print("Recall: ", recall)
print("F1: ", f1)

In [ ]:


# create a random forest classifier
rf = RandomForestClassifier(n_estimators=500, random_state=4, max_depth=300, criterion='gini', bootstrap=True)
rf_selected = RandomForestClassifier(n_estimators=500, random_state=4, max_depth=300, criterion='gini', bootstrap=True)
# use the list
rf_probeset_names = RandomForestClassifier(n_estimators=500, random_state=4, max_depth=300, criterion='gini', bootstrap=True)

# fit the model
rf.fit(train_data_final, y_train)
rf_selected.fit(train_data_selected, y_train)
rf_probeset_names.fit(train_data.loc[:, probeset_names], y_train)

# evaluate the model
accuracy = rf.score(valid_data_final, y_valid)
accuracy_selected = rf_selected.score(valid_data_selected, y_valid)
accuracy_probeset_names = rf_probeset_names.score(valid_data.loc[:, probeset_names], y_valid)

# shapes


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# Validation  

In [ ]:
# get the predictions
predictions = rf.predict(test_data_final)
predictions_selected = rf_selected.predict(test_data_selected)
predictions_probeset_names = rf_probeset_names.predict(test_data.loc[:, probeset_names])

# get the accuracy
accuracy = accuracy_score(test_data['state'], predictions)
accuracy_selected = accuracy_score(test_data['state'], predictions_selected)
accuracy_probeset_names = accuracy_score(test_data['state'], predictions_probeset_names)

# get the recall
recall = recall_score(test_data['state'], predictions)
recall_selected = recall_score(test_data['state'], predictions_selected)
recall_probeset_names = recall_score(test_data['state'], predictions_probeset_names)

# get the f1 score
f1 = f1_score(test_data['state'], predictions)
f1_selected = f1_score(test_data['state'], predictions_selected)
f1_probeset_names = f1_score(test_data['state'], predictions_probeset_names)

print("Accuracy: ", accuracy)

print("Accuracy selected: ", accuracy_selected)

print("Accuracy probeset names: ", accuracy_probeset_names)

print("Recall: ", recall)

print("Recall selected: ", recall_selected)

print("Recall probeset names: ", recall_probeset_names)

print("F1: ", f1)

print("F1 selected: ", f1_selected)

print("F1 probeset names: ", f1_probeset_names)

print("Number of selected genes: ", len(selected_features_final))

print("Number of selected genes: ", len(selected_features))

print("Number of selected genes: ", len(probeset_names))

# get the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test_data['state'], predictions)

# get the feature importances
feature_importances = rf.feature_importances_

# get the indices of the features sorted by importance
indices = np.argsort(feature_importances)[::-1]

# get the names of the features
names = [selected_features_final[i] for i in indices]

# plot the feature importances (show only the top 20)
plt.figure()
plt.title("Feature importances")
plt.bar(range(20), feature_importances[indices][:20])
plt.xticks(range(20), names[:20], rotation=90)
plt.show()

#plt.bar(range(len(selected_features_final)), feature_importances[indices])

#plt.xticks(range(len(selected_features_final)), names, rotation=90)
plt.show()


